<div style="border:solid green 2px; padding: 20px">
<b>Айбек, привет!</b>

Меня зовут Евгений Д., и я буду проверять твой проект. Предлагаю общаться на «ты» 😊 Но если это не удобно - дай знать, и мы перейдем на "вы". 

Моя основная цель — не указать на совершенные тобою ошибки, а поделиться своим опытом и помочь тебе стать аналитиком данных. Ты уже проделал большую работу над проектом, но давай сделаем его еще лучше. Ниже ты найдешь мои комментарии - пожалуйста, не перемещай, не изменяй и не удаляй их. Я буду использовать цветовую разметку:

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b> Так выделены самые важные замечания. Без их отработки проект не будет принят. </div>

<div class="alert alert-warning">
<b>Комментарий ревьюера:</b> Так выделены небольшие замечания. Одна-два таких замечания в проекте допустимы, но если их будет больше - тебе будет необходимо внести исправления. Это как тестовое задание при приеме на работу: очень много мелких ошибок могут стать причиной отказа кандидату. 

</div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b> Так я выделяю все остальные комментарии.</div>

Давай работать над проектом в диалоге: если ты что-то меняешь в проекте или отвечаешь на мои комменатри — пиши об этом. Мне будет легче отследить изменения, если ты выделишь свои комментарии:
<div class="alert alert-info"> <b>Комментарий студента:</b> Например, вот так.</div>

Всё это поможет выполнить повторную проверку твоего проекта оперативнее. 

## Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Здорово, когда есть вступление в сам проект, каждый сможет быстрее разобраться с тем о чем сам проект. Можно также привести краткий план проведения самой работы и описание используемых столбцов. 

</div>

### Шаг 1. Откройте файл с данными и изучите общую информацию. 

In [1]:
import pandas as pd
from pymystem3 import Mystem
from nltk.stem import SnowballStemmer 
from collections import Counter

In [2]:
df = pd.read_csv('/datasets/data.csv')

In [3]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       19351 non-null float64
dob_years           21525 non-null int64
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        19351 non-null float64
purpose             21525 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB
None


 1. days_employed и total_income имеют пропущенные значения
 2. days_employed и total_income должны быть числовыми
 3. Возможно пустые поля взаимосвязаны

In [5]:
df[(df['total_income'].isnull() == True) & (df['days_employed'].isnull() == True)].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2174 entries, 12 to 21510
Data columns (total 12 columns):
children            2174 non-null int64
days_employed       0 non-null float64
dob_years           2174 non-null int64
education           2174 non-null object
education_id        2174 non-null int64
family_status       2174 non-null object
family_status_id    2174 non-null int64
gender              2174 non-null object
income_type         2174 non-null object
debt                2174 non-null int64
total_income        0 non-null float64
purpose             2174 non-null object
dtypes: float64(2), int64(5), object(5)
memory usage: 220.8+ KB


Так и есть, недавно утроившиеся сотрудники - не получали зарплату

In [6]:
df.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21525.000000,19351.000000,21525.000000,21525.000000,21525.000000,21525.000000,1.935100e+04
mean,0.538908,63046.497661,43.293380,0.817236,0.972544,0.080883,1.674223e+05
std,1.381587,140827.311974,12.574584,0.548138,1.420324,0.272661,1.029716e+05
min,-1.000000,-18388.949901,0.000000,0.000000,0.000000,0.000000,2.066726e+04
25%,0.000000,-2747.423625,33.000000,1.000000,0.000000,0.000000,1.030532e+05
50%,0.000000,-1203.369529,42.000000,1.000000,0.000000,0.000000,1.450179e+05
75%,1.000000,-291.095954,53.000000,1.000000,1.000000,0.000000,2.034351e+05
max,20.000000,401755.400475,75.000000,4.000000,4.000000,1.000000,2.265604e+06


1. min children имеет значение -1, скорее всего нужно заменить на 0
2. max children имеет значение 20, трудно в это верится, скорее всего просто 2
3. отрциательные знаечения days_employed - скорее всего надо взять по модулю
4. mean days_employed 63046 дней - слишком большое число 

In [7]:
print('Кол-во с -1 ребенком:', df[df['children']==-1].count()[0])
print('Кол-во с 20 ребенком:', df[df['children']==20].count()[0])
print("Кол-во 'days_employed' > 0:", df[df['days_employed'] > 0].shape[0])
print("Кол-во 'days_employed' < 0:", df[df['days_employed'] < 0].shape[0])


Кол-во с -1 ребенком: 47
Кол-во с 20 ребенком: 76
Кол-во 'days_employed' > 0: 3445
Кол-во 'days_employed' < 0: 15906


### Вывод

1. Кол-во с ребенком -1 надо исправить
2. Кол-во с ребенком 20 тоже надо исправить 
3. Отрицательный days_employed просто возьмем по модулю 

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Достаточно детально изучили наши первичные данные, давай теперь их обработаем, ведь у нас есть план 😊 
</div>

### Шаг 2. Предобработка данных

### Обработка пропусков

Заполним пустые значения средним значением соответсвующего столбца

In [9]:
df['days_employed'] = df.groupby('income_type')['days_employed'].transform(lambda x: x.fillna(x.mean()))
df['total_income'] = df.groupby('income_type')['total_income'].transform(lambda x: x.fillna(x.mean()))
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Отличный способ, точечно увеличить нашу дисперсию 😊
    
Кстати, почему выбрали именно средние значения ? А не медиану например. 
    
</div>

Возьмем days_employed и total_income по модулю и переведем в числовое значние

In [10]:
df[['total_income', 'days_employed']] = df[['total_income', 'days_employed']].abs()

children -1 заменим на 0
children 20 заменим на 2

In [11]:
df['children'] = df['children'].replace(-1, 0)
df['children'] = df['children'].replace(20, 2)

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

А если бы мы посмотрели на долю этих значений, то смогли бы принять решение в сторону избавления от этих данных ? 
    
</div>

### Замена типа данных

In [12]:
df['days_employed'] = df['days_employed'].astype('int')
df['total_income'] = df['total_income'].astype('int')

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе метод `astype` и `fillna` можно применять к нескольким столбцам одновременно.
    
Если столбцов более 2–ух, то рекомендую применять цикл `for`.
    
</div>

In [13]:
df.dtypes

children             int64
days_employed        int64
dob_years            int64
education           object
education_id         int64
family_status       object
family_status_id     int64
gender              object
income_type         object
debt                 int64
total_income         int64
purpose             object
dtype: object

### Вывод

days_displayed и total_income перевели в int

### Вывод

### Обработка дубликатов

In [14]:
df.duplicated().sum()

54

In [15]:
df[df.duplicated(keep=False)].sort_values(by=['total_income', 'days_employed'])

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
1005,0,365003,62,среднее,1,женат / замужем,0,F,пенсионер,0,137127,ремонт жилью
1191,0,365003,61,среднее,1,женат / замужем,0,F,пенсионер,0,137127,операции с недвижимостью
1511,0,365003,58,высшее,0,Не женат / не замужем,4,F,пенсионер,0,137127,дополнительное образование
1681,0,365003,57,среднее,1,гражданский брак,1,F,пенсионер,0,137127,на проведение свадьбы
2052,0,365003,58,среднее,1,гражданский брак,1,F,пенсионер,0,137127,сыграть свадьбу
...,...,...,...,...,...,...,...,...,...,...,...,...
10697,0,2111,40,среднее,1,гражданский брак,1,F,компаньон,0,202417,сыграть свадьбу
13878,1,2111,31,среднее,1,женат / замужем,0,F,компаньон,0,202417,покупка жилья
17379,0,2111,54,высшее,0,женат / замужем,0,M,компаньон,0,202417,операции с коммерческой недвижимостью
17774,1,2111,40,среднее,1,гражданский брак,1,F,компаньон,0,202417,строительство жилой недвижимости


In [16]:
df = df.drop_duplicates()

In [17]:
df.duplicated().sum()

0

### Вывод

Выявили дубликаты и удалили их

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Хорошо, что удалили дубликаты, они нам ни к чему.
    
</div>

### Лемматизация

Выведем список уникальных purpose

In [18]:
purposes = df['purpose'].value_counts().index.tolist()
print(df['purpose'].value_counts().to_frame())

                                        purpose
свадьба                                     793
на проведение свадьбы                       773
сыграть свадьбу                             769
операции с недвижимостью                    675
покупка коммерческой недвижимости           662
операции с жильем                           652
покупка жилья для сдачи                     652
операции с коммерческой недвижимостью       650
покупка жилья                               646
жилье                                       646
покупка жилья для семьи                     638
строительство собственной недвижимости      635
недвижимость                                633
операции со своей недвижимостью             627
строительство жилой недвижимости            625
покупка недвижимости                        621
покупка своего жилья                        620
строительство недвижимости                  619
ремонт жилью                                607
покупка жилой недвижимости              

Судя по списку выше:
1. Надо убрать "покупка" - общее понятие
2. Надо убрать "строительство" - связано с жильем
3. Надо убрать "операция" - связано с жильем

In [23]:
from nltk.stem import SnowballStemmer 
russian_stemmer = SnowballStemmer('russian')
m = Mystem()
def do_lemma(row):
    lemma = m.lemmatize(row)
    return lemma

In [24]:
df['purpose_lemma'] = df['purpose'].apply(do_lemma)

<div class="alert alert-danger">
<b>Комментарий ревьюера:</b>

<s>Поправь пожалуйста ошибку в коде, чтобы я смог проверить проект дальше.
    
Рекомендую перед отправкой проекта запустить весь проект целиком, чтобы не возникло дополнительных ошибок.
    
</div>

<div class="alert alert-info"> <b>Комментарий студента:</b> Поправил, учту в дальнейшем </div>

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

👍
    
</div>

In [25]:
all_lemmas_list = df['purpose_lemma'].values
lemmas_list = []
for sublist in all_lemmas_list:
    for item in sublist:
        lemmas_list.append(item)

In [26]:
print(Counter(lemmas_list))

Counter({' ': 33596, '\n': 21471, 'недвижимость': 6353, 'покупка': 5900, 'жилье': 4461, 'автомобиль': 4308, 'образование': 4014, 'с': 2918, 'операция': 2604, 'свадьба': 2335, 'свой': 2231, 'на': 2228, 'строительство': 1879, 'высокий': 1374, 'получение': 1315, 'коммерческий': 1312, 'для': 1290, 'жилой': 1231, 'сделка': 941, 'дополнительный': 907, 'заниматься': 904, 'проведение': 773, 'сыграть': 769, 'сдача': 652, 'семья': 638, 'собственный': 635, 'со': 627, 'ремонт': 607, 'подержанный': 486, 'подержать': 478, 'приобретение': 461, 'профильный': 436})


Создадим словарь для самых топовых, без учета пробелов и других не имеющих значение слов

In [27]:
def dictionary(world, data):
    for row in data:
        if world == row:
            return row

In [28]:
dict = []
dict.append(dictionary('недвижимость', lemmas_list))
dict.append(dictionary('жилье', lemmas_list))
dict.append(dictionary('автомобиль', lemmas_list))
dict.append(dictionary('образование', lemmas_list))
dict.append(dictionary('свадьба', lemmas_list))
print(dict)

['недвижимость', 'жилье', 'автомобиль', 'образование', 'свадьба']


### Вывод

Выбрали топ целей для которых берется кредит

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Категории кредитов опредилили верно 👍
    
</div>

### Категоризация данных

Разделим по категориям цели кредитования

In [29]:
def category(data):
    if dict[0] in data:
        return dict[0]
    elif dict[1] in data:
        return dict[1]
    elif dict[2] in data:
        return dict[2]
    elif dict[3] in data:
        return dict[3]
    else:
        return dict[4]

In [30]:
df['purpose_category'] = df['purpose_lemma'].apply(category)
print(df.loc[:, ['purpose', 'purpose_category']] .head(15))

                              purpose purpose_category
0                       покупка жилья            жилье
1             приобретение автомобиля       автомобиль
2                       покупка жилья            жилье
3          дополнительное образование      образование
4                     сыграть свадьбу          свадьба
5                       покупка жилья            жилье
6                   операции с жильем            жилье
7                         образование      образование
8               на проведение свадьбы          свадьба
9             покупка жилья для семьи            жилье
10               покупка недвижимости     недвижимость
11  покупка коммерческой недвижимости     недвижимость
12                    сыграть свадьбу          свадьба
13            приобретение автомобиля       автомобиль
14         покупка жилой недвижимости     недвижимость


Категории по days_employed и children

In [31]:
def income_category(row):
    if row['total_income'] == 0:
        return '0'
    elif row['total_income'] <= 20000:
        return '0-20'
    elif row['total_income'] <= 50000:
        return '20-50'
    elif row['total_income'] <= 100000:
        return '50-100'
    elif row['total_income'] <= 250000:
        return '100-250'
    elif row['total_income'] <= 500000:
        return '250-500'
    else:
        return '500+'
    
def days_employed_category(row):
    if row['days_employed'] <= 3652:
        return 'стаж до 10 лет'
    elif 3652 < row['days_employed'] <= 6904:
        return 'стаж 10-30 лет'
    else:
        return 'стаж более 30 лет'

def children_category(row):
    if row['children'] == 0:
        return 'нет детей'
    elif 1 <= row['children'] <= 2:
        return '1-2 ребенка'
    else:
        return 'многодетные'

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В том числе, чтобы определить категории по нашим доходам, мы можем обратиться к стандартным квантилям.
    
</div>

In [32]:
df['income_category'] = df.apply(income_category, axis=1)
df['days_employed_category'] = df.apply(days_employed_category, axis=1)
df['children_category'] = df.apply(children_category, axis=1)

In [33]:
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,purpose_lemma,purpose_category,income_category,days_employed_category,children_category
0,1,8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]",жилье,250-500,стаж более 30 лет,1-2 ребенка
1,1,4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль,100-250,стаж 10-30 лет,1-2 ребенка
2,0,5623,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]",жилье,100-250,стаж 10-30 лет,нет детей
3,3,4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]",образование,250-500,стаж 10-30 лет,многодетные
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба,100-250,стаж более 30 лет,нет детей


### Вывод

Разделили по категориям цели, для которых берется кредит

### Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [34]:
df.groupby('children_category')['debt'].mean()

children_category
1-2 ребенка    0.093124
многодетные    0.081579
нет детей      0.075173
Name: debt, dtype: float64

### Вывод

Люди без детей более ответственны в вопросе выплаты кредитов

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
</div>

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [35]:
df.groupby('family_status')['debt'].mean().sort_values()

family_status
вдовец / вдова           0.065693
в разводе                0.071130
женат / замужем          0.075421
гражданский брак         0.093202
Не женат / не замужем    0.097509
Name: debt, dtype: float64

### Вывод

Люди, которые не были в браке, склонны к долгам

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
Обрати внимание, что наши действия повторяются с точки зрения расчетов.
    
Это говорит о том, что мы можем создать функцию для постоянных расчетов. Тем самым мы сможем оптимизировать свой код 😊
    
</div>

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [36]:
df.groupby('income_category')['debt'].mean().sort_values()

income_category
20-50      0.061828
500+       0.063063
250-500    0.069471
50-100     0.080909
100-250    0.084044
Name: debt, dtype: float64

### Вывод

Заемщики с малым и самым большим доходом, не склонны к долгам

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Верно 👍
    
В том числе можно было посчитать с помощью сводных таблиц `pivot_table`
    
</div>

- Как разные цели кредита влияют на его возврат в срок?

In [37]:
df.groupby('purpose_category')['debt'].mean().sort_values()

purpose_category
жилье           0.069043
недвижимость    0.074610
свадьба         0.079657
образование     0.092177
автомобиль      0.093547
Name: debt, dtype: float64

### Вывод

Заемщики, которые берут кредит для покупки жилья, менее склонны к долгам

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

В точку 👍
    
В том числе можно было бы построить графики, например с помощью метода `hist` 
    
Как раз визуализацию уже будем проходить в следующем проекте. 
    
</div>

### Шаг 4. Общий вывод

Из вышеуказанного видно, что более надежными заемщиками являются люди, состоящие в браке, имеют заработную плату до 50 и боле 250, и с целью приобретения жилья

<div class="alert alert-success">
<b>Комментарий ревьюера:</b>

Выводы понятны и логичны, а самое главное подкреплены выявленными фактами. Финальный вывод мы можем писать более объемно (по желанию), но самое конечно главное, чтобы он содержал в себе ответ на главный вопрос в задаче, что мы собственно и сделали 😊
       
В целом сам проект выполнен на хорошем уровне! На протяжении всей работы чувствуется глубина проработки задачи и это безусловный плюс для нас. Было использовано большое количество методов, которые будут помогать тебе в дальнейших проектах.
    
Не стоит забывать про оформление своей работы, это важный такой же важный этап. Рекомендую «не стесняться» использовать ячейки типа **markdown** там где нам необходимо отобразить свои размышления, в том числе с использованием дополнительной стилизации.
    
Поздравляю тебя с успешной сдачей проекта и желаею успехов в новых работах! 
    
</div>

### Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [ ]  файл изучен;
- [ ]  определены пропущенные значения;
- [ ]  заполнены пропущенные значения;
- [ ]  есть пояснение, какие пропущенные значения обнаружены;
- [ ]  описаны возможные причины появления пропусков в данных;
- [ ]  объяснено, по какому принципу заполнены пропуски;
- [ ]  заменен вещественный тип данных на целочисленный;
- [ ]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [ ]  удалены дубликаты;
- [ ]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [ ]  описаны возможные причины появления дубликатов в данных;
- [ ]  выделены леммы в значениях столбца с целями получения кредита;
- [ ]  описан процесс лемматизации;
- [ ]  данные категоризированы;
- [ ]  есть объяснение принципа категоризации данных;
- [ ]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [ ]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [ ]  в каждом этапе есть выводы;
- [ ]  есть общий вывод.